# Lab 3.2 : PySpark CORE APIs

In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()

## Step 1 - Load sample file

In [ ]:
f = spark.read.text("/data/text/twinkle/sample.txt")
print (f)

After executing the above...  

**=> Goto Spark shell UI (4040+)**  
**=> Inspect the 'Jobs'  and 'Stages' section in the UI.**  
**=> How is the filter executed? Can you explain the behavior?**  


In [ ]:
print(f.count())
f.show()

**==> Now do you see jobs executed in the Spark UI?**

## Step 2 - Filter

Note we are referring to 'value'

In [ ]:
# both f.value and f["value"] will work
# later version recommended

filtered = f.filter(f.value.contains("twinkle"))
filtered = f.filter(f["value"].contains("twinkle"))

**==> Are filters executed yet?**

In [ ]:
print(filtered.count())
filtered.show()

**==>How about now?**

## Step 3 - See the DAG visualizations

<img src="../assets/images/3.1c.png" style="border: 5px solid grey; max-width:100%;"/>


## Step 4 - Generate Large Dataset

In [ ]:
! echo "creating data..."

! [ ! -r /data/text/twinkle/500M.data ] && cd  /data/text/twinkle &&   ./create-data-files.sh

! echo "DONE"

! ls -lSrh /data/text/twinkle
# sorted by size (smallest --> largest)

## Step 5 - Load larger dataset

In [ ]:
f = spark.read.text("/data/text/twinkle/100M.data")
print (f.count())
f.show()

## Step 6 - Inspect Job Details
How many tasks were allocated to the latest job?  Can you figure out why?  
Hint : get number of partitions

In [ ]:
f.rdd.getNumPartitions()

## Step 7 - Save filtered data

In [ ]:
f = spark.read.text("/data/text/twinkle/100M.data")
filtered = f.filter(f["value"].contains("twinkle"))
filtered.write.mode('overwrite').text("out2")

In [ ]:
## inspect the output dir
! ls -lh out2

As the program is running checkout spark console (where you started Jupyter from).
You will see output like this

[Stage 16:==========================>                             (15 + 4) / 32]

Also inspect 'out' directory.
What do you see?

## Step 8 - Bonus  -  Merging multiple partitions into ONE
When we saved data in the above section, there are multiple files created in output directory. Can you just create one output file?

Hint : see the API for `coalesce` or `repartition`